In [1]:
import numpy as np
from scipy.stats import poisson, nbinom, norm, multivariate_normal, truncnorm
from numpy.linalg import cholesky, LinAlgError

def nb_theta_mu_to_r_p(theta, mu):
    r = theta
    p = theta / (theta + mu)
    return r, p

def cdf_poisson(k, mu):
    return poisson.cdf(k, mu)

def cdf_nb_theta_mu(k, theta, mu):
    r, p = nb_theta_mu_to_r_p(theta, mu)
    return nbinom.cdf(k, r, p)

def cdf_zip(k, mu, pi):
    if k < 0:
        return 0.0
    if k == 0:
        return pi + (1 - pi) * poisson.pmf(0, mu)
    return pi + (1 - pi) * poisson.cdf(k, mu)

def cdf_zinb_theta_mu(k, theta, mu, pi):
    r, p = nb_theta_mu_to_r_p(theta, mu)
    if k < 0:
        return 0.0
    if k == 0:
        return pi + (1 - pi) * nbinom.pmf(0, r, p)
    return pi + (1 - pi) * nbinom.cdf(k, r, p)

def ghk_estimate_rectangle_prob(a, b, Sigma, M=10000):
    """
    Estimate P(a < Z < b) for Z ~ N(0, Sigma) using GHK.
    a, b: arrays of length p (lower, upper)
    Sigma: correlation matrix (p x p)
    M: number of simulation draws
    Returns: estimate of probability
    """
    p = Sigma.shape[0]
    # Cholesky: Sigma = L L^T
    try:
        L = cholesky(Sigma)
    except LinAlgError:
        raise ValueError("Sigma is not positive definite")
    # We simulate M samples
    weights = np.zeros(M)
    for m in range(M):
        # sequential draw
        z = np.zeros(p)
        w = 1.0
        for i in range(p):
            # compute conditional mean and variance of z_i given previous z[0:i]
            # because z = L * u, u ~ N(0,I). So we can simulate u sequentially.
            mu_cond = np.dot(L[i, :i], z[:i])
            sigma_cond = L[i, i]
            # compute lower and upper truncation in u-space
            a_u = (a[i] - mu_cond) / sigma_cond
            b_u = (b[i] - mu_cond) / sigma_cond
            # truncated normal draw for u_i
            u_i = truncnorm.rvs(a_u, b_u, loc=0.0, scale=1.0)
            # weight contribution for this step
            w *= (norm.cdf(b_u) - norm.cdf(a_u))
            # set z_i = mu_cond + sigma_cond * u_i
            z[i] = mu_cond + sigma_cond * u_i
        weights[m] = w
    return weights.mean()

def gaussian_copula_point_probability_ghk(x, marg_params, Sigma,
                                           eps=1e-12, M=5000):
    """
    Estimate Pr(X == x) under Gaussian copula + discrete marginals,
    using GHK simulation with M draws.
    """
    x = np.asarray(x, dtype=int)
    p = x.shape[0]
    # verify/correct Sigma to correlation matrix (like earlier)
    diag = np.sqrt(np.diag(Sigma))
    R = Sigma / np.outer(diag, diag)
    np.fill_diagonal(R, 1.0)

    # compute a_i, b_i
    a = np.empty(p)
    b = np.empty(p)
    for i, xi in enumerate(x):
        params = marg_params[i]
        if params[0] == 0:
            # non zero-inflated
            if params[1] == np.inf:
                mu = params[2]
                a[i] = cdf_poisson(xi - 1, mu)
                b[i] = cdf_poisson(xi, mu)
            else:
                theta = params[1]; mu = params[2]
                a[i] = cdf_nb_theta_mu(xi - 1, theta, mu)
                b[i] = cdf_nb_theta_mu(xi, theta, mu)
        else:
            # zero-inflated
            pi = params[0]
            if params[1] == np.inf:
                mu = params[2]
                a[i] = cdf_zip(xi - 1, mu, pi)
                b[i] = cdf_zip(xi, mu, pi)
            else:
                theta = params[1]; mu = params[2]
                a[i] = cdf_zinb_theta_mu(xi - 1, theta, mu, pi)
                b[i] = cdf_zinb_theta_mu(xi, theta, mu, pi)
        a[i] = np.clip(a[i], eps, 1 - eps)
        b[i] = np.clip(b[i], eps, 1 - eps)

    # convert to z-space limits
    z_lower = norm.ppf(a)
    z_upper = norm.ppf(b)

    # Estimate via GHK: P(z_lower < Z < z_upper)
    prob = ghk_estimate_rectangle_prob(z_lower, z_upper, R, M=M)
    return prob

def getPointScores(
    targetAD, 
    auxMargParams, auxCovMat, auxCopulaGenes, 
    synthMargParams, synthCovMat, synthCopulaGenes
):
    scores = {}
    for i, cell in enumerate(targetAD):
        print(i)
        try:
            auxExpr = cell[:,auxCopulaGenes].X.toarray().flatten()
            auxProb = gaussian_copula_point_probability_ghk(
                auxExpr, auxMargParams, auxCovMat, M=100
            )
            synthExpr = cell[:,synthCopulaGenes].X.toarray().flatten()
            synthProb = gaussian_copula_point_probability_ghk(
                synthExpr, synthMargParams, synthCovMat, M=100
            )
            scores[i] = auxProb / synthProb
        except:
            scores[i] = None
            
    return scores

In [53]:
import numpy as np
import torch
from scipy.stats import poisson, nbinom
from numpy.linalg import cholesky, LinAlgError
from math import sqrt

# ---------- Marginal CDF helpers (vectorized via scipy) ----------
def nb_theta_mu_to_r_p(theta, mu):
    r = theta
    p = theta / (theta + mu)
    return r, p

def cdf_poisson_vec(k_arr, mu_arr):
    # k_arr and mu_arr are numpy arrays (same shape) or broadcastable
    # For k < 0 returns 0 automatically via scipy
    return poisson.cdf(k_arr, mu_arr)

def cdf_nb_theta_mu_vec(k_arr, theta_arr, mu_arr):
    r_arr, p_arr = theta_arr, theta_arr / (theta_arr + mu_arr)
    # scipy.nbinom.cdf accepts arrays for k, n (r), p
    return nbinom.cdf(k_arr, r_arr, p_arr)

def cdf_zip_vec(k_arr, mu_arr, pi_arr):
    # pi in [0,1]
    # vectorized: handle k<0 and k==0 properly
    k_arr = np.asarray(k_arr)
    mu_arr = np.asarray(mu_arr)
    pi_arr = np.asarray(pi_arr)
    out = np.zeros_like(mu_arr, dtype=float)
    mask_neg = k_arr < 0
    out[mask_neg] = 0.0
    mask_zero = (k_arr == 0)
    if np.any(mask_zero):
        out[mask_zero] = pi_arr[mask_zero] + (1 - pi_arr[mask_zero]) * poisson.pmf(0, mu_arr[mask_zero])
    mask_pos = k_arr > 0
    if np.any(mask_pos):
        out[mask_pos] = pi_arr[mask_pos] + (1 - pi_arr[mask_pos]) * poisson.cdf(k_arr[mask_pos], mu_arr[mask_pos])
    return out

def cdf_zinb_theta_mu_vec(k_arr, theta_arr, mu_arr, pi_arr):
    k_arr = np.asarray(k_arr)
    theta_arr = np.asarray(theta_arr)
    mu_arr = np.asarray(mu_arr)
    pi_arr = np.asarray(pi_arr)
    out = np.zeros_like(mu_arr, dtype=float)
    mask_neg = k_arr < 0
    out[mask_neg] = 0.0
    mask_zero = (k_arr == 0)
    if np.any(mask_zero):
        r = theta_arr[mask_zero]
        p = theta_arr[mask_zero] / (theta_arr[mask_zero] + mu_arr[mask_zero])
        out[mask_zero] = pi_arr[mask_zero] + (1 - pi_arr[mask_zero]) * nbinom.pmf(0, r, p)
    mask_pos = k_arr > 0
    if np.any(mask_pos):
        r = theta_arr[mask_pos]
        p = theta_arr[mask_pos] / (theta_arr[mask_pos] + mu_arr[mask_pos])
        out[mask_pos] = pi_arr[mask_pos] + (1 - pi_arr[mask_pos]) * nbinom.cdf(k_arr[mask_pos], r, p)
    return out

# ---------- Torch Gaussian helpers (cdf/icdf) ----------
_torch_sqrt2 = np.sqrt(2.0)

def torch_norm_cdf(x):
    # x: torch tensor (double)
    # Φ(x) = 0.5 * (1 + erf(x / sqrt(2)))
    return 0.5 * (1.0 + torch.erf(x / _torch_sqrt2))

def torch_norm_icdf(u):
    # inverse cdf using erfinv: Φ^{-1}(u) = sqrt(2) * erfinv(2u - 1)
    # u in (0,1)
    return _torch_sqrt2 * torch.erfinv(2.0 * u - 1.0)

# ---------- GHK on GPU, batched over B samples and M draws ----------
def ghk_estimate_rectangle_prob_torch_batch(z_lower, z_upper, R, M=10000, device=None, eps=1e-14):
    """
    Vectorized GHK estimator on GPU using PyTorch.

    Inputs:
      z_lower: numpy array shape (B, p)
      z_upper: numpy array shape (B, p)
      R: correlation matrix (p, p) numpy array
      M: number of Monte Carlo draws
      device: 'cuda' or 'cpu' (if None, auto select)
    Returns:
      probs: numpy array shape (B,) estimated probabilities for each sample
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device(device)

    z_lower = np.asarray(z_lower, dtype=float)
    z_upper = np.asarray(z_upper, dtype=float)
    R = np.asarray(R, dtype=float)

    B, p = z_lower.shape
    # Cholesky of R (correlation -> positive definite check)
    try:
        L = cholesky(R)
    except LinAlgError as e:
        raise ValueError("R is not positive definite") from e
    # convert to torch on device
    L_t = torch.as_tensor(L, dtype=torch.float64, device=device)  # p x p
    z_lower_t = torch.as_tensor(z_lower, dtype=torch.float64, device=device)  # B x p
    z_upper_t = torch.as_tensor(z_upper, dtype=torch.float64, device=device)  # B x p

    # We'll perform M draws per sample in parallel: tensors of shape (B, M)
    # Initialize log-weights for each (B,M)
    logw = torch.zeros((B, M), dtype=torch.float64, device=device)

    # z_i storage: we only need values up to current i
    # We'll keep z_prev as tensor of shape (i, B, M) initially empty
    z_prev = None  # will be a tensor (i, B, M)

    sqrt2 = _torch_sqrt2

    for i in range(p):
        # compute mu_cond = L[i, :i] @ z_prev (for each sample and each draw)
        if i == 0:
            mu_cond = torch.zeros((B, M), dtype=torch.float64, device=device)
        else:
            # z_prev: (i, B, M), L_row (i,)
            L_row = L_t[i, :i]  # vector length i
            # tensordot over axis 0 of z_prev (the gene axis) with L_row
            # result shape (B, M)
            mu_cond = torch.tensordot(L_row, z_prev, dims=([0], [0]))  # (B, M)

        sigma_cond = float(L_t[i, i].item())  # scalar

        # compute truncation on standard normal (u-space):
        a_u = (z_lower_t[:, i:i+1] - mu_cond) / sigma_cond  # (B, M) broadcasting
        b_u = (z_upper_t[:, i:i+1] - mu_cond) / sigma_cond

        # compute Phi(a_u), Phi(b_u)
        Phi_a = torch_norm_cdf(a_u)
        Phi_b = torch_norm_cdf(b_u)

        # numerical safety: clamp differences away from 0
        diff = (Phi_b - Phi_a).clamp(min=eps)

        # sample uniform in [Phi_a, Phi_b] for each (B,M)
        U = torch.rand((B, M), dtype=torch.float64, device=device)
        U = Phi_a + U * (Phi_b - Phi_a)

        # invert to get standard normal truncated sample u_i
        u_i = torch_norm_icdf(U)

        # compute z_i = mu_cond + sigma_cond * u_i
        z_i = mu_cond + sigma_cond * u_i  # (B, M)

        # append z_i to z_prev
        if z_prev is None:
            z_prev = z_i.unsqueeze(0)  # shape (1, B, M)
        else:
            z_prev = torch.cat([z_prev, z_i.unsqueeze(0)], dim=0)  # shape (i+1, B, M)

        # update log-weights
        logw = logw + torch.log(diff)

    # weights: exp(logw) (shape B x M), average over M draws
    # To avoid overflow/underflow we can use log-sum-exp trick per row:
    # prob_b = mean_m exp(logw[b,m]) = exp(logsumexp(logw[b,:]) - log(M))
    # implement vectorized:
    max_logw, _ = logw.max(dim=1, keepdim=True)  # (B,1)
    exp_shift = torch.exp(logw - max_logw)  # (B,M)
    sum_exp = exp_shift.sum(dim=1)  # (B,)
    probs_t = (sum_exp * torch.exp(max_logw.squeeze(1))) / float(M)  # (B,)

    probs = probs_t.cpu().numpy()
    return probs

# ---------- Build marginal a/b arrays for a batch of cells ----------
def build_ab_for_batch(x_batch, marg_params, eps=1e-12):
    """
    x_batch: numpy array shape (B,p) integer counts
    marg_params: list length p where each element is (pi, theta, mu)
      - pi==0 means not zero-inflated, theta==np.inf means Poisson
      - same convention you used
    Returns:
      a_batch, b_batch: numpy arrays shape (B, p)
    """
    x_batch = np.asarray(x_batch, dtype=int)
    B, p = x_batch.shape
    a = np.empty((B, p), dtype=float)
    b = np.empty((B, p), dtype=float)

    # Vectorized over genes (loop over p is fine if p moderate)
    for j in range(p):
        params = marg_params[j]
        xj = x_batch[:, j]
        if params[0] == 0:
            # not zero-inflated
            if params[1] == np.inf:
                mu = np.full(B, params[2], dtype=float)
                a[:, j] = cdf_poisson_vec(xj - 1, mu)
                b[:, j] = cdf_poisson_vec(xj, mu)
            else:
                theta = np.full(B, params[1], dtype=float)
                mu = np.full(B, params[2], dtype=float)
                a[:, j] = cdf_nb_theta_mu_vec(xj - 1, theta, mu)
                b[:, j] = cdf_nb_theta_mu_vec(xj, theta, mu)
        else:
            pi = np.full(B, params[0], dtype=float)
            if params[1] == np.inf:
                mu = np.full(B, params[2], dtype=float)
                a[:, j] = cdf_zip_vec(xj - 1, mu, pi)
                b[:, j] = cdf_zip_vec(xj, mu, pi)
            else:
                theta = np.full(B, params[1], dtype=float)
                mu = np.full(B, params[2], dtype=float)
                a[:, j] = cdf_zinb_theta_mu_vec(xj - 1, theta, mu, pi)
                b[:, j] = cdf_zinb_theta_mu_vec(xj, theta, mu, pi)
        # clip
        a[:, j] = np.clip(a[:, j], eps, 1 - eps)
        b[:, j] = np.clip(b[:, j], eps, 1 - eps)
    return a, b

# ---------- High-level vectorized probability function ----------
def gaussian_copula_point_probability_ghk_batch(
    X_batch, marg_params, Sigma, M=5000, device=None
):
    """
    X_batch: numpy array shape (B, p) with integer counts (each row is a cell)
    marg_params: list length p (per-gene) as before
    Sigma: covariance (p,p) - will be converted to correlation R
    Returns:
      probs: numpy array shape (B,)
    """
    # convert Sigma to correlation matrix R (like original)
    diag = np.sqrt(np.diag(Sigma))
    R = Sigma / np.outer(diag, diag)
    np.fill_diagonal(R, 1.0)

    # Build a and b (cdf bounds)
    a_batch, b_batch = build_ab_for_batch(X_batch, marg_params)

    # z-limits
    # use the normal inverse cdf on CPU (numpy) then push to torch inside ghk function
    from scipy.stats import norm as sp_norm
    z_lower = sp_norm.ppf(a_batch)
    z_upper = sp_norm.ppf(b_batch)

    # call GHK sampler (torch)
    probs = ghk_estimate_rectangle_prob_torch_batch(z_lower, z_upper, R, M=M, device=device)
    return probs

# ---------- Batched getPointScores (works with your AnnData-like cells) ----------
def getPointScores_gpu(
    targetAD,
    auxMargParams, auxCovMat, auxCopulaGenes,
    synthMargParams, synthCovMat, synthCopulaGenes,
    batch_size=32, M=2000, device=None
):
    """
    targetAD: iterable of cell slices (like your original code where each
      'cell' is subsetting an AnnData row: cell[:, genes].X)
      It's assumed each cell row produces a (1 x p) sparse matrix accessible by:
          arr = cell[:, gene_indices].X.toarray().flatten()
      If targetAD is a dense numpy array (n_cells x n_genes) you can adapt easily.
    batch_size: number of cells processed together
    M: MC draws for GHK
    device: cuda/cpu (default auto)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # helper to extract a dense 1D array from the cell selection
    def extract_row(cell, genes):
        # keep original approach for compatibility
        arr = cell[:, genes].X
        # If it's sparse or dense, ensure numpy 1-D
        if hasattr(arr, "toarray"):
            arr = arr.toarray().flatten()
        else:
            arr = np.asarray(arr).reshape(-1)
        return arr.astype(int)

    # gather all cells into a list for indexing
    n_cells = len(targetAD)
    scores = {}
    idx = 0
    while idx < n_cells:
        print(f"{idx} / {n_cells}")
        # build batch
        batch_cells = []
        batch_indices = list(range(idx, min(idx + batch_size, n_cells)))
        for i in batch_indices:
            try:
                r = extract_row(targetAD[i], auxCopulaGenes)
            except Exception:
                r = None
            batch_cells.append(r)
        # split into aux and synth arrays (some may be None)
        # For those None, we will set result to None and skip heavy work
        valid_mask = [row is not None for row in batch_cells]
        if not any(valid_mask):
            for ii, i_cell in enumerate(batch_indices):
                scores[i_cell] = None
            idx += batch_size
            continue

        # build X_aux_batch and X_synth_batch arrays for valid rows
        valid_rows_idx = [ii for ii, ok in enumerate(valid_mask) if ok]
        X_aux = np.vstack([batch_cells[ii] for ii in valid_rows_idx])  # shape (Bval, p_aux)
        # For synth we need to extract synth genes per the same indices in targetAD
        X_synth = np.vstack([
            extract_row(targetAD[batch_indices[ii]], synthCopulaGenes)
            for ii in valid_rows_idx
        ])

        # compute probs in batch (aux and synth)
        try:
            aux_probs = gaussian_copula_point_probability_ghk_batch(
                X_aux, auxMargParams, auxCovMat, M=M, device=device
            )
        except Exception as e:
            aux_probs = np.full((len(valid_rows_idx),), np.nan)
            print("Error computing aux_probs for batch starting at", idx, ":", e)

        try:
            synth_probs = gaussian_copula_point_probability_ghk_batch(
                X_synth, synthMargParams, synthCovMat, M=M, device=device
            )
        except Exception as e:
            synth_probs = np.full((len(valid_rows_idx),), np.nan)
            print("Error computing synth_probs for batch starting at", idx, ":", e)

        # assign back to scores
        vi = 0
        for ii, ok in enumerate(valid_mask):
            global_idx = batch_indices[ii]
            if not ok:
                scores[global_idx] = None
            else:
                a_p = aux_probs[vi]
                s_p = synth_probs[vi]
                # guard against division by zero/nan
                if (s_p == 0) or np.isnan(a_p) or np.isnan(s_p):
                    scores[global_idx] = None
                else:
                    scores[global_idx] = float(a_p / s_p)
                vi += 1
        idx += batch_size
    return scores

In [46]:
scoresGPU = getPointScores_gpu(
    adata_sample,
    auxCopulaMarginals, auxCovMat, auxCopulaGenes, 
    synthCopulaMarginals, synthCovMat, synthCopulaGenes,
    batch_size=32, M=2000, device=None
)

In [95]:
import rdata
import pandas as pd
import scanpy as sc

synthModel = rdata.read_rds("splits/1/model/0.rds")['0']
synthCopulaGenes = synthModel['gene_sel1']
synthCovMat = synthModel['cov_mat']
synthCopulaMarginals = synthModel['marginal_param1']

auxModel = rdata.read_rds("models/0.rds")['0']
auxCopulaGenes = auxModel['gene_sel1']
auxCovMat = auxModel['cov_mat']
auxCopulaMarginals = auxModel['marginal_param1']

auxAD = sc.read_h5ad("train.h5ad")
auxCT0 = auxAD[auxAD.obs.cell_type==0]

trainAD = sc.read_h5ad("splits/1/train.h5ad")
trainInds = set(trainAD.obs.individual.unique())
all_inds = auxCT0.obs["individual"].astype(str)
actualLabels = all_inds.isin(trainInds).tolist()

# actualLabels = []
# for i, cell in enumerate(auxCT0):
#     ind = cell.obs["individual"].astype(str)[0]
#     if ind in trainInds:
#         actualLabels.append(True)
#     else:
#         actualLabels.append(False)

# idx = np.random.choice(auxCT0.n_obs, size=1000, replace=False)
# adata_sample = auxCT0[idx].copy()

# scores = getPointScores(
#     adata_sample, auxCopulaMarginals, auxCovMat, auxCopulaGenes, 
#     synthCopulaMarginals, synthCovMat, synthCopulaGenes
# )

In [57]:
scoresGPU = getPointScores_gpu(
    auxCT0,
    auxCopulaMarginals, auxCovMat, auxCopulaGenes, 
    synthCopulaMarginals, synthCovMat, synthCopulaGenes,
    batch_size=1000, M=2000, device=None
)

0 / 234731
1000 / 234731
2000 / 234731
3000 / 234731
4000 / 234731
5000 / 234731
6000 / 234731
7000 / 234731
8000 / 234731
9000 / 234731
10000 / 234731
11000 / 234731
12000 / 234731
13000 / 234731
14000 / 234731
15000 / 234731
16000 / 234731
17000 / 234731
18000 / 234731
19000 / 234731
20000 / 234731
21000 / 234731
22000 / 234731
23000 / 234731
24000 / 234731
25000 / 234731
26000 / 234731
27000 / 234731
28000 / 234731
29000 / 234731
30000 / 234731
31000 / 234731
32000 / 234731
33000 / 234731
34000 / 234731
35000 / 234731
36000 / 234731
37000 / 234731
38000 / 234731
39000 / 234731
40000 / 234731
41000 / 234731
42000 / 234731
43000 / 234731
44000 / 234731
45000 / 234731
46000 / 234731
47000 / 234731
48000 / 234731
49000 / 234731
50000 / 234731
51000 / 234731
52000 / 234731
53000 / 234731
54000 / 234731
55000 / 234731
56000 / 234731
57000 / 234731
58000 / 234731
59000 / 234731
60000 / 234731
61000 / 234731
62000 / 234731
63000 / 234731
64000 / 234731
65000 / 234731
66000 / 234731
67000 / 

In [96]:
# oldScores = scores
scores = scoresGPU

In [97]:
import random
percentile_66 = np.percentile([x for x in scores.values() if x is not None], 66)
predictedLabels = [bool(random.getrandbits(1)) for i in range(len(scores))]
for ind, score in scores.items():
    if score is not None:
        predictedLabels[ind] = score > percentile_66

In [98]:
from sklearn.metrics import roc_auc_score

auroc = roc_auc_score(actualLabels, predictedLabels)
print("AUROC:", auroc)

AUROC: 0.5399298219241364


In [29]:
import scanpy as sc
trainAD = sc.read_h5ad("splits/1/train.h5ad")

/home/lysander/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [46]:
ct0 = trainAD[trainAD.obs.cell_type==0, copulaGenes]
cell = ct0[0].X.toarray().flatten()

In [54]:
gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat)

9.530491408619522e-17

In [63]:
vals = []
for i in [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    vals.append(gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat, M=i))

In [70]:
gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat, M=100)

8.94688859631545e-17

In [78]:
for i, c in enumerate(trainAD):
    print(i)
    print(c.X.toarray().flatten())
    break

0
[0. 0. 0. ... 0. 0. 0.]
